💡 [`csr_matrix` information on indexing](https://stackoverflow.com/a/52299730) \
💡 [`scipy.sparse.coo_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy-sparse-coo-matrix) \
💡 [`scipy.sparse.csr_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy-sparse-csr-matrix) \
💡 [`scipy.sparse.lil_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html#scipy.sparse.lil_matrix) \
💡 [`%%timeit` magic documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html) 

### 0. imports

In [1]:
# data science
import scipy as sp
import numpy as np
# random numbers https://numpy.org/doc/stable/reference/random/index.html#random-quick-start
from numpy.random import default_rng
rng = default_rng()
# plotting
import matplotlib as plt
# type hints
from scipy.sparse import (
    coo_matrix,
    csr_matrix,
    csc_matrix,
    lil_matrix
)
from IPython.core.magics.execution import TimeitResult

### 1.1. generate sparse matrices

In [2]:
M_coo = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'coo',
    dtype = float,
    random_state = 42
)

In [3]:
M_csr = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'csr',
    dtype = float,
    random_state = 42
)

In [4]:
M_csc = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'csc',
    dtype = float,
    random_state = 42
)

In [5]:
M_lil = sp.sparse.random(
    m = 20000,
    n = 20000,
    density = 0.01,
    format = 'lil',
    dtype = float,
    random_state = 42
)

### 1.2. generate index for matrix

In [6]:
index_matrix: np.array = np.arange(
    start = 0,
    stop = 20000,
    step = 1,
    dtype=int
)

### 1.3. generate list of rows to set to zero

In [7]:
# contains M.shape[0] 0s and 1s of a specified ratio
# 1s will be masked and then removed below, 0s will remain untouched
array_for_masking: np.array = rng.choice(
    a = [0, 1],
    size = (M_coo.shape[0], ),
    p = [0.2, 0.8]
)

# contains only indices of rows to be set to zero 
index_matrix_zero: np.array = np.ma.masked_array(
    data = index_matrix,
    mask = array_for_masking
).compressed()

In [8]:
index_matrix_zero.shape

(4025,)

### 2.1. setting rows to zero
#### 2.1.1 ["COO: multiply with a diagonal matrix"](https://stackoverflow.com/a/65364784)

In [9]:
def matrix_multiplication(
    matrix: coo_matrix,
    rows: np.array
) -> coo_matrix:

    array_for_diagonal: np.array = np.ones(
        shape = [matrix.shape[0]],
        dtype = int
    )
    array_for_diagonal[rows] = 0

    matrix = matrix @ sp.sparse.diags(array_for_diagonal)

    return matrix


#### 2.1.2 ["CSR: edit `csr.data` using a for-loop with `csr.indptr`"](https://stackoverflow.com/q/12129948)

💡 [`csr_matrix` information on indexing](https://stackoverflow.com/a/52299730)

In [10]:
def csr_loop_set_rows_to_zero(
    matrix: csr_matrix,
    rows: np.array
) -> csr_matrix:

    for row in rows:
        matrix.data[
            matrix.indptr[row]:matrix.indptr[row+1]
        ] = 0
        
    return matrix

#### 2.1.3 ["CSR: regular indexing notation"](https://stackoverflow.com/a/15900629)

⚠️ this runs too long for `%%timeit` to complete in a reasonable time

In [11]:
def csr_indexing_set_rows_to_zero(
    matrix: csr_matrix,
    rows: np.array
) -> csr_matrix:

    for row in rows:
        matrix[row,:] = 0
    
    return matrix

#### 2.1.4 ["LIL: regular indexing notation"](https://stackoverflow.com/a/69310639)

💡 [`scipy.sparse.lil_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html#scipy.sparse.lil_matrix)

In [12]:
def lil_set_rows_to_zero(
    matrix: lil_matrix,
    rows: np.array
) -> lil_matrix:

    for row in rows:
        matrix[row,:] = 0
    
    return matrix

#### 2.1.5 "CSR/dense: convert back-and-forth between dense and sparse matrices"

In [20]:
def convert_to_dense_and_back(
    matrix: coo_matrix,
    rows: np.array
) -> coo_matrix:

    mask_rows = np.ones(
        shape = (matrix.shape[0], ),
        dtype = bool
    )
    mask_rows[rows] = False

    matrix_dense = matrix.todense()
    matrix_dense[mask_rows] = 0
    matrix = coo_matrix(matrix_dense)

    return matrix


#### 2.1.6 ["CSR: mask out `csr.data` and `csr.indices` using `csr.indptr`"](https://stackoverflow.com/a/19800305)

❓ what does `np.diff(csr_matrix.indptr)` do? \
❗️ it counts the number of `data` per *index pointer* (`indptr`), which is the number of `data` per *row*

❓ what does `np.repeat(mask, nnz_per_row)` do? \
❗️ see OneNote drawing

In [14]:
def csr_mask_set_rows_to_zero(
    matrix: csr_matrix,
    rows: np.array
) -> csr_matrix:

    number_nonzero_per_row: np.array = np.diff(matrix.indptr)

    mask_rows = np.ones(
        shape = (matrix.shape[0], ),
        dtype = bool
    )
    mask_rows[rows_to_zero] = False

    mask = np.repeat(
        a = mask_rows, # input array, TRUE for rows to keep, FALSE for rows to set 0
        repeats = number_nonzero_per_row # array with number of non-zero elements (=data) per row
    )
 
    number_nonzero_per_row[rows_to_zero] = 0

    matrix.data = matrix.data[mask]
    matrix.indices = matrix.indices[mask]
    matrix.indptr[1:] = np.cumsum(number_nonzero_per_row)

    return matrix

#### 2.1.7 ["CSR: concatenate magic"](https://stackoverflow.com/a/58748646)

🙈 whaaat?

In [15]:
def qq():
    idx = a.indptr.searchsorted(*(a.indices==i).nonzero(),"right")-1
    return np.bincount(
        np.concatenate([a.indices[a.indptr[i]:a.indptr[i+1]] for i in idx]),
        np.concatenate([a.data[a.indptr[i]:a.indptr[i+1]] for i in idx]),
        a.shape[1]) / len(idx)

#### 2.1.8 [`pylcaio` implementation (convert to `pd.DataFrame`)](https://github.com/OASES-project/pylcaio/blob/master/src/pylcaio.py#L1340)

In [ ]:
"""
def pylcaio_implementation(
    matrix: csr_matrix,
    rows: np.array
) -> np.array:

    # sets all dataframe rows to zero where the process ID matches the one in list_to_hyb 
    ANT.loc[self.list_to_hyb] = 0
    ANT.fillna(0,inplace=True)

    # matrix of production processes
    Amarket = self.A_ff.copy()
    Amarket = pd.DataFrame(
        data = Amarket.todense(),
        index = self.PRO_f.index,
        columns = self.PRO_f.index
    )

    # sets all dataframe rows to zero where the process ID matches the one in listmarket 
    Amarket.loc[self.listmarket] = 0
    Amarket.fillna(0,inplace=True)
"""

### 3.1 measuring performance

In [16]:
time_lil: TimeitResult =  %timeit -o lil_set_rows_to_zero(M_lil, index_matrix_zero)

1.44 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
time_csr_data: TimeitResult =  %timeit -o csr_loop_set_rows_to_zero(M_csr, index_matrix_zero)

2.02 ms ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
time_coo: TimeitResult = %timeit -o matrix_multiplication(M_coo, index_matrix_zero)

176 ms ± 729 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
time_conversion: TimeitResult = %timeit -o convert_to_dense_and_back(M_coo, index_matrix_zero)

2.22 s ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
time_csr_mask: TimeitResult = %timeit -o csr_mask_set_rows_to_zero(M_csr, index_matrix_zero)

10.5 ms ± 36.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
